In [22]:
import pandas as pd
import random

In [19]:
df_a = pd.read_csv('attraction_info_final_1217.csv')
df_r = pd.read_csv('place_info_final_1217.csv')

In [16]:
maping = {
    '士林區': 1,
    '北投區': 1,
    '文山區': 2,
    '內湖區': 3,
    '南港區': 3,
    '中山區': 4,
    '松山區': 4,
    '信義區': 4,
    '大安區': 4,
    '萬華區': 5,
    '中正區': 5,
    '大同區': 5
}

df_a['district_num'] = df_a['new_district'].map(maping)
df_r['district_num'] = df_r['new_district'].map(maping)
df_a.to_csv('attraction_info_final_1217.csv', index=False)
df_r.to_csv('place_info_final_1217.csv', index=False)

# 根據權重選區

In [266]:
df_a = pd.read_csv('attraction_info_final_1217.csv')
df_r = pd.read_csv('place_info_final_1217.csv')

In [267]:
df_a_nightview = df_a[df_a['new_place_category'] == '夜景']
df_a_nightmarket = df_a[df_a['new_place_category'] == '夜市']
df_a2 = df_a.drop(df_a_nightview.index)
df_a2 = df_a.drop(df_a_nightmarket.index)

In [268]:
district = ["1", "2", "3", "4", "5"]
# Weights for each item
weights = [0.15, 0.1, 0.15, 0.35, 0.25]
# Select a random item from the list, with the weights specified
selected_district = random.choices(district, weights=weights)[0]

# 抽取景點行程1、2

In [269]:
# 建立空的 DataFrame 存放最終行程順序
result = pd.DataFrame()

# 依據ratings欄位建立weights欄位

In [288]:
# 選取被挑選出的欄位
df_a2 = df_a2[df_a2['district_num'] == int(selected_district)]

# 將rating大於4.3的*2, 小於3.7的/2 存到 rating2
df_a2['rating2'] = df_a2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))

# 
df_a2['weights'] = df_a2['rating2'] / df_a2['rating2'].sum()

In [289]:
# 隨機取樣一行
attraction_1 = df_a2.sample(n=1, weights=df_a2['weights'])
# result = pd.concat([result, attraction_1])

# 從原始df_a2中移除attraction_1的行
df_a2 = df_a2.drop(attraction_1.index)

# 隨機取樣一行
attraction_2 = df_a2.sample(n=1, weights=df_a2['weights'])
# result = pd.concat([result, attraction_2])

In [290]:
d_latitude = (attraction_1['latitude'].values - attraction_2['latitude'].values)[0]
d_longitude = (attraction_1['longitude'].values - attraction_2['longitude'].values)[0]
print(d_latitude)
print(d_longitude)

0.018846800000002162
-0.015824100000003227


In [291]:
print(attraction_1['latitude'])
print(attraction_2['latitude'])

202    25.032131
Name: latitude, dtype: float64
9    25.013284
Name: latitude, dtype: float64


In [292]:
if abs(d_latitude) > abs(d_longitude):
    # 緯度差距大於精度差距，以緯度分割
    if d_latitude > 0:
        # 如果attraction_1在attraction_2的上面
        
        # 早餐店要篩選latitude > attraction_1的latitude
        filt_r_1 = (df_r['new_place_category'] == '早午餐') & (df_r['latitude'] > attraction_1['latitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_r_1 = df_r[filt_r_1]
        if df_r_1.shape[0] != 0:
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
        else:
            filt_r_1 = df_r['district_num'] == int(selected_district)
            df_r_1 = df_r[filt_r_1]
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
        
        # category_list 接收回傳值
        category_list = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        category_all = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        
        # 午餐要選在attraction_1['latitude'] attraction_2['latitude']中間，且符合使用者選擇的類別
        filt_r_2 = df_r[df_r['new_place_category'].isin(category_list)]
        filt_r_2 = filt_r_2[attraction_1['latitude'].values[0]  > filt_r_2['latitude']]
        filt_r_2 = filt_r_2[attraction_2['latitude'].values[0]  < filt_r_2['latitude']]
        df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
        df_r_2.shape[0]
        
        if df_r_2.shape[0] != 0:
            # 如果有在中間
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一間餐廳
            filt_r_2 = df_r[df_r['new_place_category'].isin(category_all)]
            df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
            
            
        # 景點3要篩選latitude < attraction_2的latitude
        filt_a_3 = (df_a2['latitude'] < attraction_2['latitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_a_3 = df_r[filt_a_3]
        
        if df_a_3.shape[0] != 0:
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一個景點
            filt_a_3 = df_a2['district_num'] == int(selected_district)
            df_a_3 = df_a2[filt_a_3]
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
        
    else:
        # 如果attraction_1在attraction_2的下面
        # 早餐店要篩選latitude < attraction_1的latitude
        filt_r_1 = (df_r['new_place_category'] == '早午餐') & (df_r['latitude'] < attraction_1['latitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_r_1 = df_r[filt_r_1]
        if df_r_1.shape[0] != 0:
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
        else:
            filt_r_1 = df_r['district_num'] == int(selected_district)
            df_r_1 = df_r[filt_r_1]
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
            
        # category_list 接收回傳值
        category_list = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        category_all = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        
        # 午餐要選在attraction_1['latitude'] attraction_2['latitude']中間，且符合使用者選擇的類別
        filt_r_2 = df_r[df_r['new_place_category'].isin(category_list)]
        filt_r_2 = filt_r_2[attraction_1['latitude'].values[0]  < filt_r_2['latitude']]
        filt_r_2 = filt_r_2[attraction_2['latitude'].values[0]  > filt_r_2['latitude']]
        df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
        df_r_2.shape[0]
        
        if df_r_2.shape[0] != 0:
            # 如果有在中間
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一間餐廳
            filt_r_2 = df_r[df_r['new_place_category'].isin(category_all)]
            df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
            
            
        # 景點3要篩選latitude > attraction_2的latitude
        filt_a_3 = (df_a2['latitude'] > attraction_2['latitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_a_3 = df_r[filt_a_3]
        
        if df_a_3.shape[0] != 0:
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一個景點
            filt_a_3 = df_a2['district_num'] == int(selected_district)
            df_a_3 = df_a2[filt_a_3]
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
            
            
else:
    # 經度差距大於緯度差距，以經度分割
    if d_longitude > 0:
        # 如果attraction_1在attraction_2的右邊
        # 早餐店要篩選longitude > attraction_1的longitude
        filt_r_1 = (df_r['new_place_category'] == '早午餐') & (df_r['longitude'] > attraction_1['longitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_r_1 = df_r[filt_r_1]
        if df_r_1.shape[0] != 0:
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
        else:
            filt_r_1 = df_r['district_num'] == int(selected_district)
            df_r_1 = df_r[filt_r_1]
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
            
        # category_list 接收回傳值
        category_list = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        category_all = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        
        # 午餐要選在attraction_1['longitude'] attraction_2['longitude']中間
        filt_r_2 = df_r[df_r['new_place_category'].isin(category_list)]
        filt_r_2 = filt_r_2[attraction_1['longitude'].values[0]  > filt_r_2['longitude']]
        filt_r_2 = filt_r_2[attraction_2['longitude'].values[0]  < filt_r_2['longitude']]
        df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
        df_r_2.shape[0]
        
        if df_r_2.shape[0] != 0:
            # 如果有在中間
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一間餐廳
            filt_r_2 = df_r[df_r['new_place_category'].isin(category_all)]
            df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
            
            
        # 景點3要篩選longitude < attraction_2的longitude
        filt_a_3 = (df_a2['longitude'] < attraction_2['longitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_a_3 = df_r[filt_a_3]
        
        if df_a_3.shape[0] != 0:
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一個景點
            filt_a_3 = df_a2['district_num'] == int(selected_district)
            df_a_3 = df_a2[filt_a_3]
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
            
    else:
        # 如果attraction_1在attraction_2的左邊
        # 早餐店要篩選longitude < attraction_1的longitude
        filt_r_1 = (df_r['new_place_category'] == '早午餐') & (df_r['longitude'] < attraction_1['longitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_r_1 = df_r[filt_r_1]
        if df_r_1.shape[0] != 0:
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
        else:
            filt_r_1 = df_r['district_num'] == int(selected_district)
            df_r_1 = df_r[filt_r_1]
            df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
            restaurant_1 = df_r_1.sample(n=1, weights=df_r_1['weights'])
            
        # category_list 接收回傳值
        category_list = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        category_all = ['中式','韓式','台灣小吃/熱炒店','異國料理','港式', '意式', '燒烤店', '南洋', '美式', '火鍋', '素食']
        
        
        # 午餐要選在attraction_1['longitude'] attraction_2['longitude']中間
        filt_r_2 = df_r[df_r['new_place_category'].isin(category_list)]
        filt_r_2 = filt_r_2[attraction_1['longitude'].values[0]  < filt_r_2['longitude']]
        filt_r_2 = filt_r_2[attraction_2['longitude'].values[0]  > filt_r_2['longitude']]
        df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
        df_r_2.shape[0]
        
        if df_r_2.shape[0] != 0:
            # 如果有在中間
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一間餐廳
            filt_r_2 = df_r[df_r['new_place_category'].isin(category_all)]
            df_r_2 = filt_r_2[filt_r_2['district_num'] == int(selected_district)]
            df_r_2['rating2'] = df_r_2['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_r_2['weights'] = df_r_2['rating2'] / df_r_1['rating2'].sum()
            restaurant_2 = df_r_2.sample(n=1, weights=df_r_2['weights'])
        # 景點3要篩選longitude > attraction_2的longitude
        filt_a_3 = (df_a2['longitude'] > attraction_2['longitude'].values[0]) & (df_r['district_num'] == int(selected_district))
        df_a_3 = df_r[filt_a_3]
        
        if df_a_3.shape[0] != 0:
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])
        else:
            # 如果不符合上述任一條件，就在該區隨機挑選一個景點
            filt_a_3 = df_a2['district_num'] == int(selected_district)
            df_a_3 = df_a2[filt_a_3]
            df_a_3['rating2'] = df_a_3['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
            df_a_3['weights'] = df_a_3['rating2'] / df_a_3['rating2'].sum()
            attraction_3 = df_a_3.sample(n=1, weights=df_a_3['weights'])

C:\Users\jimmy\AppData\Local\Temp\ipykernel_17560\2770064694.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_r_1['rating2'] = df_r_1['total_rating'].apply(lambda x: x*2 if x >= 4.3 else (x/2 if x <= 3.7 else x))
C:\Users\jimmy\AppData\Local\Temp\ipykernel_17560\2770064694.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_r_1['weights'] = df_r_1['rating2'] / df_r_1['rating2'].sum()
C:\Users\jimmy\AppData\Local\Temp\ipykernel_17560\2770064694.py:40: SettingWithCopyWarning: 
A value is trying to 

In [293]:
restaurant_1

,google_url,place_name,total_rating,place_category,total_reviews,cost,address,district,eat_in,to_go_1,...,thursday,friday,saturday,sunday,new_place_category,longitude,latitude,district_num,rating2,weights
8219,https://www.google.com.tw/maps/place/OJ+Mornin...,OJ Morning 早午餐,4.8,早午餐餐廳,220.0,NaN,"號1樓, No. 116昌吉街大同區台北市103",Plus Code: 3G87+8R 大同區 台北市,1,1,...,"[0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....",早午餐,25.065867,121.514578,5.0,9.6,0.004891


In [294]:
attraction_1

,google_url,place_name,total_rating,place_category,total_reviews,cost,address,district,eat_in,to_go_1,...,sunday,new_district,new_rating,new_review,latitude,longitude,new_place_category,district_num,rating2,weights
202,https://www.google.com.tw/maps/place/%E6%A5%8A...,楊英風美術館,4.4,藝術博物館,91.0,NaN,100台北市中正區重慶南路二段31號,Plus Code: 2GJ7+VQ 中正區 台北市,0,0,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",中正區,4,51-200,25.032131,121.514497,藝術中心,5,8.8,0.016188


In [295]:
restaurant_2

,google_url,place_name,total_rating,place_category,total_reviews,cost,address,district,eat_in,to_go_1,...,thursday,friday,saturday,sunday,new_place_category,longitude,latitude,district_num,rating2,weights
13825,https://www.google.com.tw/maps/place/%E5%AE%8F...,宏益水晶餃,3.9,餃子店,575.0,$,108台北市萬華區昆明街184之2號,Plus Code: 2GR3+JQ 萬華區 台北市,1,0,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",中式,25.041605,121.504454,5.0,3.9,0.001987


In [296]:
attraction_2

,google_url,place_name,total_rating,place_category,total_reviews,cost,address,district,eat_in,to_go_1,...,sunday,new_district,new_rating,new_review,latitude,longitude,new_place_category,district_num,rating2,weights
9,https://www.google.com.tw/maps/place/%E8%87%AA...,自來水園區,4.3,主題公園/遊樂場,1804.0,NaN,100台北市中正區思源街1號,Plus Code: 2G7J+84 中正區 台北市,0,0,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0...",中正區,4,1001-2000,25.013284,121.530321,主題公園/遊樂場,5,8.6,0.01582


In [297]:
attraction_3

,google_url,place_name,total_rating,place_category,total_reviews,cost,address,district,eat_in,to_go_1,...,sunday,new_district,new_rating,new_review,latitude,longitude,new_place_category,district_num,rating2,weights
197,https://www.google.com.tw/maps/place/%E6%9D%B1...,東和禪寺,4.2,佛寺,299.0,NaN,100台北市中正區仁愛路一段21之33號,Plus Code: 2GQF+P4 中正區 台北市,0,0,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",中正區,4,201-500,25.039313,121.522836,寺廟/教堂,5,4.2,0.007853


In [298]:
result = pd.concat([result, restaurant_1])
result = pd.concat([result, attraction_1])
result = pd.concat([result, restaurant_2])
result = pd.concat([result, attraction_2])
result = pd.concat([result, attraction_3])


# 最後合併完 reset index
result = result.reset_index(drop=True)

In [287]:
result

,google_url,place_name,total_rating,place_category,total_reviews,cost,address,district,eat_in,to_go_1,...,thursday,friday,saturday,sunday,new_place_category,longitude,latitude,district_num,rating2,weights
0,https://www.google.com.tw/maps/place/%E5%98%89...,嘉味軒早餐店,4.1,早餐餐廳,168.0,$,100台北市中正區寧波西街226號,Plus Code: 2GG6+P9 中正區 台北市,1,1,...,"[0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",早午餐,25.026872,121.510949,5.0,4.1,0.002089
1,https://www.google.com.tw/maps/place/%E5%8F%B0...,台灣基督長老教會-李春生紀念教堂,4.0,長老教會教堂,101.0,NaN,103台北市大同區貴德街44號,Plus Code: 3G45+27 大同區 台北市,0,0,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",寺廟/教堂,121.508245,25.055108,5.0,4.0,0.007305
2,https://www.google.com.tw/maps/place/%E7%A6%8F...,福州乾拌麵,3.9,麵店,802.0,$,100台北市中正區青島東路33-3號,Plus Code: 2GVF+7X 中正區 台北市,1,1,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1....",台灣小吃/熱炒店,25.043220,121.524951,5.0,3.9,0.001987
3,https://www.google.com.tw/maps/place/%E8%8F%AF...,華中河濱公園,4.1,公園,4459.0,NaN,108台北市萬華區,Plus Code: 2F8V+5W 萬華區 台北市,0,0,...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",公園,121.494766,25.015403,5.0,4.1,0.007487
4,https://www.google.com.tw/maps/place/%E6%96%B0...,新芳春茶行（古蹟製茶廠）,4.5,歷史建築,1808.0,NaN,103台北市大同區民生西路309號,Plus Code: 3G47+R2 大同區 台北市,0,0,...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, ...",歷史景點,121.512548,25.057081,5.0,9.0,0.015357
